In [1]:
from match_compos.GUI_pair import GUIPair
import cv2
from matplotlib import pyplot as plt
from random import randint as rint
from difflib import SequenceMatcher

In [2]:
gui = GUIPair('0001')

In [3]:
gui.component_detection(False, False, False)

In [4]:
gui.show_detection_result()

No detection result, run component_detection() or load_detection_result() first


In [5]:
gui.load_detection_result()

In [6]:
gui.show_detection_result()

In [7]:
gui.save_clips()

In [7]:
gui.compos_mapping['a4c'].draw_compo(gui.img_android.copy(), show=True)
gui.compos_mapping['i7c'].draw_compo(gui.img_ios.copy(), show=True)

In [19]:
a4c = gui.compos_mapping['a4c'].clip.copy()
i7c = gui.compos_mapping['i7c'].clip.copy()

In [8]:
def dhash(image):
    image = cv2.resize(image, (9, 8), interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    dhash_str = ''
    for i in range(8):
        for j in range(8):
            if gray[i, j] > gray[i, j + 1]:
                dhash_str = dhash_str + '1'
            else:
                dhash_str = dhash_str + '0'
    #     result = ''
    #     for i in range(0, 64, 4):
    #         result += ''.join('%x' % int(dhash_str[i: i + 4], 2))
    # print("dhash值",result)
    return dhash_str


def comp_hash(hash1, hash2):
    n = 0
    if len(hash1) != len(hash2):
        return -1
    for i in range(len(hash1)):
        if hash1[i] != hash2[i]:
            n = n + 1
    return 1 - n / len(hash1)


def comp_compo_clip(img1, img2):
    h1 = dhash(img1)
    h2 = dhash(img2)
    return comp_hash(h1, h2)

In [9]:
compo_pairs = []
for compo_android in gui.compos_android:
    for compo_ios in gui.compos_ios:
        similarity = comp_compo_clip(compo_android.clip, compo_ios.clip)
        if similarity > 0.75:
            compo_pairs.append((compo_android.id, compo_ios.id))

In [10]:
text_pair = []
for compo_android in gui.compos_android:
    if compo_android.category != 'Text':
        continue
    for compo_ios in gui.compos_ios:
        if compo_ios.category != 'Text':
            continue
        text_sim = SequenceMatcher(None, compo_android.text_content, compo_ios.text_content).ratio()
        if text_sim > 0.8:
            text_pair.append((compo_android.id, compo_ios.id))

In [12]:
compo_pair = []
for compo_android in gui.compos_android:
    if compo_android.category != 'Compo':
        continue
    for compo_ios in gui.compos_ios:
        if compo_ios.category != 'Compo':
            continue
        compo_sim = comp_compo_clip(compo_android.clip, compo_ios.clip)
        if compo_sim > 0.8:
            compo_pair.append((compo_android.id, compo_ios.id))

In [15]:
board_android = gui.img_android.copy()
board_ios = gui.img_ios.copy()

In [ ]:
for pair in compo_pairs:
    color = (rint(0,255), rint(0,255), rint(0,255))
    gui.compos_mapping[pair[0]].draw_compo(board_android, color=color, line=-1, show_id=False)
    gui.compos_mapping[pair[1]].draw_compo(board_ios, color=color, line=-1, show_id=False)
    
for pair in text_pair:
    color = (rint(0,255), rint(0,255), rint(0,255))
    gui.compos_mapping[pair[0]].draw_compo(board_android, color=color, line=-1, show_id=False)
    gui.compos_mapping[pair[1]].draw_compo(board_ios, color=color, line=-1, show_id=False)
cv2.imshow('android', cv2.resize(board_android, (int(board_android.shape[1] * (800 / board_android.shape[0])), 800)))
cv2.imshow('andorid', cv2.resize(board_ios, (int(board_ios.shape[1] * (800 / board_ios.shape[0])), 800)))
cv2.waitKey()
cv2.destroyAllWindows()